In [ ]:
from pathlib import Path
PLOT_DIR = Path('plots')
PLOT_DIR.mkdir(exist_ok=True)
(PLOT_DIR / '.gitignore').write_text('*')

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns
import seaborn.objects as so
from seaborn import axes_style, move_legend
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
names=["_1", "_2", "_3", "_4", "K", "Degree", "Probability"]
df = pd.concat([pd.read_csv(f, names=names) for f in glob("data/saved/degree-*.csv")], ignore_index=True)
df

In [ ]:
p = (
    so.Plot(
        data=df[df["Degree"] <= 8000],
        # data=df,
        x="Degree", y="Probability",
        color="K",
    )
    .theme({**axes_style("ticks")})
    .scale(
        y="log",
        color=so.Continuous().tick(at=df["K"].unique()),
    )
    .add(so.Line())
)
p

In [ ]:
names=["_1", "_2", "_3", "Target Redundancy", "K", "Number of Data", "_Node Id", "Load"]
df = pd.concat([pd.read_csv(f, names=names) for f in glob("data/saved/load-*.csv")], ignore_index=True)
df

In [ ]:
sns.displot(
    df[df["Number of Data"] == 400], 
    kind="ecdf",
    x="Load", hue="Target Redundancy",
)

In [ ]:
sns.displot(
    df[df["Target Redundancy"] == 1.6], 
    kind="ecdf",
    x="Load", hue="Number of Data",
)

In [ ]:
names=["_1", "Enter Rate", "Leave Rate", "Target Redundancy", "K", "Scale Estimation Interval", "_7", "Number of Data", "Step", "Number of Node", "Redundancy"]
df = pd.concat([pd.read_csv(f, names=names) for f in glob("data/checkpoint-*.csv")], ignore_index=True)
df

In [ ]:
p = (
    so.Plot(
        data=df,
        x="Step", y="Number of Node",
    )
    .theme({**axes_style("ticks")})
    .add(so.Line())
)
p

In [ ]:
p = (
    so.Plot(
        data=df,
        x="Step", y="Redundancy",
    )
    .theme({**axes_style("ticks")})
    .add(so.Line())
)
p